# `Sal.json` Processing 

In [10]:
import pandas as pd
import re

Previwe sal.json file

In [11]:
df = pd.read_json("../data/sal.json", orient="index")
df.reset_index(names="location", inplace=True)
df.tail()

,location,ste,gcc,sal
15335,christmas island,9,9oter,90001
15336,home island,9,9oter,90002
15337,jervis bay,9,9oter,90003
15338,norfolk island,9,9oter,90004
15339,west island,9,9oter,90005


In [12]:
# convert data type for each feature
df.ste = df.ste.astype("int8")
df.sal = df.ste.astype("int16")
df.dtypes

location    object
ste           int8
gcc         object
sal          int16
dtype: object

In [14]:
# drop any rural sal value, this won't be use in the future
df = df[~df.gcc.str.contains(r"\dr[a-z]{3}")]
df = df[~df.gcc.str.contains("9oter")]
df.tail()

# keep for future usage if we do need to count rural tweets

,location,ste,gcc,sal
15330,weetangera,8,8acte,8
15331,weston (act),8,8acte,8
15332,whitlam,8,8acte,8
15333,wright,8,8acte,8
15334,yarralumla,8,8acte,8


In [15]:
df.gcc.unique()

array(['1gsyd', '2gmel', '3gbri', '4gade', '5gper', '6ghob', '7gdar',
       '8acte'], dtype=object)

In [16]:
df.shape

(3389, 4)

In [6]:
# Count number of index with state identifier, remove brackets around state abbreviations.
df[df.location.str.contains("[()]")]

,location,ste,gcc,sal
1,abbotsford (nsw),1,1gsyd,1
7,alison (central coast - nsw),1,1gsyd,1
12,annandale (nsw),1,1gsyd,1
14,appin (nsw),1,1gsyd,1
15,arcadia (nsw),1,1gsyd,1
...,...,...,...,...
15311,reid (act),8,8acte,8
15316,spence (act),8,8acte,8
15317,stirling (act),8,8acte,8
15322,theodore (act),8,8acte,8


In [35]:
# replace the () with an empty string
df.location = df.agg(lambda x: re.sub(r"[()]", "", x.location), axis=1)
df[df.location.str.contains("[()]")]

,location,ste,gcc,sal


In [37]:
# count number of index contains ' - '
df[df.location.str.contains(" - ")]

,location,ste,gcc,sal
7,alison central coast - nsw,1,1gsyd,1
198,colo hawkesbury - nsw,1,1gsyd,1
230,darlington sydney - nsw,1,1gsyd,1
255,dural hornsby - nsw,1,1gsyd,1
274,elderslie camden - nsw,1,1gsyd,1
...,...,...,...,...
10562,west end townsville - qld,3,3rqld,3
10586,white rock cairns - qld,3,3rqld,3
10660,wooroonooran cairns - qld,3,3rqld,3
10661,wooroonooran cassowary coast - qld,3,3rqld,3


In [39]:
# replace " - " with " "
df.location = df.agg(lambda x: re.sub(" - ", " ", x.location), axis=1)
df[df.location.str.contains(" - ")]

,location,ste,gcc,sal


In [40]:
# count number of index contains "."
df[df.location.str.contains("\.")]

,location,ste,gcc,sal
4543,abbotsford vic.,2,2gmel,2
4548,albert park vic.,2,2gmel,2
4549,albion vic.,2,2gmel,2
4551,altona vic.,2,2gmel,2
4555,armadale vic.,2,2gmel,2
...,...,...,...,...
14875,windermere tas.,6,6rtas,6
14880,woodbridge tas.,6,6rtas,6
14881,woodbury tas.,6,6rtas,6
14883,woodstock tas.,6,6rtas,6


In [42]:
# replace "\." with ""
df.location = df.agg(lambda x: re.sub("\.", "", x.location), axis=1)
df[df.location.str.contains("\.")]

,location,ste,gcc,sal


In [43]:
df.value_counts("gcc")

gcc
1rnsw    3623
3rqld    2587
2rvic    2373
5rwau    1303
4rsau    1202
1gsyd     920
6rtas     656
3gbri     647
2gmel     572
4gade     494
5gper     397
7rnte     202
8acte     137
6ghob     120
7gdar     102
9oter       5
dtype: int64

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15340 entries, 0 to 15339
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   location  15340 non-null  object
 1   ste       15340 non-null  int8  
 2   gcc       15340 non-null  object
 3   sal       15340 non-null  int16 
dtypes: int16(1), int8(1), object(2)
memory usage: 284.8+ KB


In [51]:
gcc_dict = dict(zip(df.gcc.unique(), [g[2::] for g in df.gcc.unique()]))

df['location_x'] = df.agg(lambda x: x.location + ' ' + gcc_dict[x.gcc], axis=1)
    
df.drop(['ste', 'sal'], axis=1, inplace=True)
df

,location,gcc,location_x
0,abbotsbury,1gsyd,abbotsbury syd
1,abbotsford nsw,1gsyd,abbotsford nsw syd
2,acacia gardens,1gsyd,acacia gardens syd
3,agnes banks,1gsyd,agnes banks syd
4,airds,1gsyd,airds syd
...,...,...,...
15335,christmas island,9oter,christmas island ter
15336,home island,9oter,home island ter
15337,jervis bay,9oter,jervis bay ter
15338,norfolk island,9oter,norfolk island ter


In [46]:
df.gcc.unique()

array(['1gsyd', '1rnsw', '2gmel', '2rvic', '3gbri', '3rqld', '4gade',
       '4rsau', '5gper', '5rwau', '6ghob', '6rtas', '7gdar', '7rnte',
       '8acte', '9oter'], dtype=object)

In [ ]:
df.location = df.agg(lambda x: x.location + '', axis=1)

In [45]:
df.to_csv("../data/processed/sal.csv", index=False)

In [15]:
df = pd.read_parquet("../data/processed/sal.parquet")
df.tail()

,locat,ste,gcc,sal
15335,christmas island,9,9oter,9
15336,home island,9,9oter,9
15337,jervis bay,9,9oter,9
15338,norfolk island,9,9oter,9
15339,west island,9,9oter,9


In [92]:
from pathlib import Path
Path("./data/sal.json").parent

PosixPath('data')

In [18]:
df.gcc.unique()

array(['1gsyd', '1rnsw', '2gmel', '2rvic', '3gbri', '3rqld', '4gade',
       '4rsau', '5gper', '5rwau', '6ghob', '6rtas', '7gdar', '7rnte',
       '8acte', '9oter'], dtype=object)

In [19]:
df

,locat,ste,gcc,sal
0,abbotsbury,1,1gsyd,1
1,abbotsford nsw,1,1gsyd,1
2,acacia gardens,1,1gsyd,1
3,agnes banks,1,1gsyd,1
4,airds,1,1gsyd,1
...,...,...,...,...
15335,christmas island,9,9oter,9
15336,home island,9,9oter,9
15337,jervis bay,9,9oter,9
15338,norfolk island,9,9oter,9


In [ ]:
def process_sal(path: Path, logger: logging) -> pd.DataFrame:
    """
    Process sal.json file by removing irrelevant attributes,
    case 0: remove any gcc containing char r (r represents rural)
    case 1: remove all brackets
    case 2: remove all " - " 
    case 3: remove all "\."
    Then store the final result into a csv file.

    path (Path): root directory
    """
    logger.info("Loading sal.json into pandas")
    # load sal.json file & reset index
    df = pd.read_json(path / "data/sal.json", orient="index", columns=['gcc'])
    df = df.reset_index().rename(columns={'index': 'location'})

    # case0: drop any rural sal value, this won't be use in the future
    logger.info("Remove any location not in city")
    # df = df[~df.gcc.str.contains(r"\dr[a-z]{3}")]

    # case1: replace all brackets with an empty string
    logger.info("Substitute brackets in location")
    df.location = df.agg(lambda x: re.sub(r"[()]", "", x.location), axis=1)

    # case2: replace " - " with " "
    logger.info("Substitude string ' - ' with ' '")
    df.location = df.agg(lambda x: re.sub(" - ", " ", x.location), axis=1)

    # case3: replace "\." with ""
    logger.info("Substitude \. with an empty string")
    df.location = df.agg(lambda x: re.sub("\.", "", x.location), axis=1)

    # add a super location as a search string
    gcc_dict = dict(zip(df.gcc.unique(), [g[2::] for g in df.gcc.unique()]))
    df['location_x'] = df.agg(lambda x: x.location + ' ' + gcc_dict[x.gcc], axis=1)
    
    # store result to a csv file
    logger.info("Store sal.csv file.")
    df.to_csv("../data/processed/sal.csv", index=False)